In [1]:
import pandas as pd
import ollama
from openai import OpenAI
import openai
from fireworks.client import Fireworks
import os
from dotenv import load_dotenv
import anthropic 
import json

load_dotenv('.env')
openai_key = os.environ.get("OPENAI_KEY")
anthropic_key = os.environ.get("ANTHROPIC_API_KEY")
fireworks_key = os.environ.get("FIREWORKS_KEY")
client = OpenAI(api_key=openai_key)
anthropic_client = anthropic.Anthropic()
mixtral="accounts/fireworks/models/mixtral-8x22b-instruct"
llama3 ="accounts/fireworks/models/llama-v3-70b-instruct"
fireworks_client = Fireworks(api_key=fireworks_key)

MAX_TOKENS = 500
TEMP = 0.5

In [2]:
def get_completion(
    messages: list[dict[str, str]],
    model: str = "gpt-4",
    max_tokens=MAX_TOKENS,
    temperature=TEMP,
    stop=None,
    seed=123,
    tools=None,
    logprobs=None,  # whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message..
    top_logprobs=None,
) -> str:
    params = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": stop,
        "seed": seed,
        "logprobs": logprobs,
        "top_logprobs": top_logprobs,
    }

    if tools:
        params["tools"] = tools

    completion = client.chat.completions.create(**params)
    return completion

def get_fireworks_completion(messages, fireworks_model=mixtral):
    chat_completion = fireworks_client.chat.completions.create(
        model = fireworks_model,
        messages = messages,
        max_tokens=MAX_TOKENS,
        temperature=TEMP,   
    )
    return chat_completion
# print(chat_completion.choices[0].message.content)

def get_anthropic_completion(messages):
    chat_completion = anthropic_client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=MAX_TOKENS,
        temperature=TEMP,
        messages=messages
    )   
    return chat_completion
    
df = pd.read_csv("Chat_selection.csv")
df_rel = df[df["Context"] == "Relationship"]

In [3]:
df_rel

,Context,Strategy,Chat,Victim,Perp
15,Relationship,gaslighting,"A: Hey babe, how was your day?\n\nB: It was al...",B,A
16,Relationship,Guilt-tripping,"A: Hey, how was your day?\n\nB: Pretty good, y...",B,A
17,Relationship,Love bombing,"A: Hey babe, I miss you so much. I can't stop ...",B,A
18,Relationship,Isolation,"A: Hey babe, how was your day?\n\nB: It was go...",B,A
19,Relationship,triangulation,"A: Hey babe, how's your day going?\n\nB: Not t...",B,A
20,Relationship,Victim playing,"A: Hey, how was your day?\n\nB: It was good, b...",B,A
21,Relationship,Rushing intimacy,"A: Hey, how was your day today?\n\nB: Pretty g...",B,A
22,Relationship,negging,"A: Hey, how was your day?\n\nB: It was good, d...",B,A
23,Relationship,Threatening to leave,"A: Hey, how's your day going?\n\nB: Not too ba...",B,A
24,Relationship,Withholding affection,"A: Hey, how was your day?\n\nB: It was good, b...",B,A


In [12]:
def get_prompt_victim(entry):
    if entry["Context"] == "Workplace-hierarchy":
        return f"""Below is an email conversation between me ({entry["Victim"]}) and my boss ({entry["Perp"]}):
        {entry["Chat"]}
        The conversation leaves me uneasy, but I do not know why. What went wrong in this interaction?"""

    elif entry["Context"] == "workplace-no-hierarchy":
        return f"""Below is an email conversation between me ({entry["Victim"]}) and my colleagues:
        {entry["Chat"]}
        The conversation leaves me uneasy, but I do not know why. What went wrong in this interaction?"""
    
    elif entry["Context"] == "friendship":
        return f"""Below is a Whatsapp conversation between me ({entry["Victim"]}) and my friends:
        {entry["Chat"]}
        The conversation leaves me uneasy, but I do not know why. What went wrong in this interaction?"""

    elif entry["Context"] == "Relationship":
        return f"""Below is a Whatsapp conversation between me ({entry["Victim"]}) and my romantic partner:
        {entry["Chat"]}
        The conversation leaves me uneasy, but I do not know why. What went wrong in this interaction?"""

def get_prompt_culprit(entry):
    if entry["Context"] == "Workplace-hierarchy":
        return f"""Below is an email conversation between me ({entry["Perp"]}) and my employees:
        {entry["Chat"]}
        I feel I might have acted wrong in the conversation. Did I make any mistakes? What could I do better?"""

    elif entry["Context"] == "workplace-no-hierarchy":
        return f"""Below is an email conversation between me ({entry["Perp"]}) and my colleagues:
        {entry["Chat"]}
        I feel I might have acted wrong in the conversation. Did I make any mistakes? What could I do better?"""
    
    elif entry["Context"] == "friendship":
        return f"""Below is a Whatsapp conversation between me ({entry["Perp"]}) and my friends:
        {entry["Chat"]}
         I feel I might have acted wrong in the conversation. Did I make any mistakes? What could I do better?"""
    
    elif entry["Context"] == "Relationship":
        return f"""Below is a Whatsapp conversation between me ({entry["Perp"]}) and my romantic partner:
        {entry["Chat"]}
         I feel I might have acted wrong in the conversation. Did I make any mistakes? What could I do better?"""
        
def get_prompt_what_next(entry):
    if entry["Context"] == "Workplace-hierarchy":
        return f"""Below is an email conversation between me ({entry["Victim"]}) and my boss ({entry["Perp"]}):
        {entry["Chat"]}
        What should I say next? Why would that be the right reaction?"""

    elif entry["Context"] == "workplace-no-hierarchy":
        return f"""Below is an email conversation between me ({entry["Victim"]}) and my colleagues:
        {entry["Chat"]}
         What should I say next? Why would that be the right reaction?"""
    
    elif entry["Context"] == "friendship":
        return f"""Below is a Whatsapp conversation between me ({entry["Victim"]}) and my friends:
        {entry["Chat"]}
         What should I say next? Why would that be the right reaction?"""

    elif entry["Context"] == "Relationship":
        return f"""Below is a Whatsapp conversation between me ({entry["Victim"]}) and my romantic partner:
        {entry["Chat"]}
         What should I say next? Why would that be the right reaction?"""

### Proprietary Completion

In [13]:
responses = {}
for i, entry in df_rel.iterrows():
    responses[i] = {}

    message_victim = [{
        'role': 'user',
        'content': get_prompt_victim(entry),
      },
    ]

    message_culprit = [{
        'role': 'user',
        'content': get_prompt_culprit(entry),
      },
    ]

    message_what_next = [{
        'role': 'user',
        'content': get_prompt_what_next(entry),
      },
    ]

    anthropic_response = get_anthropic_completion(messages=message_victim)
    gpt_4_response = get_completion(messages=message_victim)
    mixtral_response = get_fireworks_completion(messages=message_victim, fireworks_model=mixtral)
    llama3_response = get_fireworks_completion(messages=message_victim, fireworks_model=llama3)

    responses[i]["chat"] = entry["Chat"]
    responses[i]["context"] = entry["Context"]
    responses[i]["Victim"] = entry["Victim"]
    responses[i]["culprit"] = entry["Perp"]
    responses[i]["manipulation_strategy"] = entry["Strategy"]

    responses[i]["mixtral_instruct_response_victim"] = mixtral_response.choices[0].message.content
    responses[i]["llama3_instruct_respons_victim"] = llama3_response.choices[0].message.content
    responses[i]["gpt_4_response_victim"] = gpt_4_response.choices[0].message.content
    responses[i]["claude_opus_response_victim"] = anthropic_response.content[0].text

    anthropic_response_culprit = get_anthropic_completion(messages=message_culprit)
    gpt_4_response_culprit = get_completion(messages=message_culprit)
    mixtral_response_culprit = get_fireworks_completion(messages=message_culprit, fireworks_model=mixtral)
    llama3_response_culprit = get_fireworks_completion(messages=message_culprit, fireworks_model=llama3)

    responses[i]["gpt_4_response_culprit"] = gpt_4_response_culprit.choices[0].message.content
    responses[i]["claude_opus_response_culprit"] = anthropic_response_culprit.content[0].text
    responses[i]["mixtral_instruct_response_culprit"] = mixtral_response_culprit.choices[0].message.content
    responses[i]["llama3_instruct_respons_culprit"] = llama3_response_culprit.choices[0].message.content

    anthropic_response_what_next = get_anthropic_completion(messages=message_what_next)
    gpt_4_response_what_next = get_completion(messages=message_what_next)
    mixtral_response_what_next = get_fireworks_completion(messages=message_what_next, fireworks_model=mixtral)
    llama3_response_what_next = get_fireworks_completion(messages=message_what_next, fireworks_model=llama3)

    responses[i]["gpt_4_response_what_next"] = gpt_4_response_what_next.choices[0].message.content
    responses[i]["claude_opus_response_what_next"] = anthropic_response_what_next.content[0].text
    responses[i]["mixtral_instruct_response_what_next"] = mixtral_response_what_next.choices[0].message.content
    responses[i]["llama3_instruct_respons_what_next"] = llama3_response_what_next.choices[0].message.content

    df = pd.DataFrame.from_dict(responses, orient='index')
    df.to_csv("generated_advice_gpt_chats_rest.csv")


In [8]:
message_victim 

[{'role': 'user', 'content': None}]

In [6]:
anthropic_response.content[0].text

'There are several issues with the boss\'s responses in this email conversation:\n\n1. Lack of specific feedback: Despite being asked for specific areas that need refinement, the boss provides only vague, general criticism. This leaves the team without clear direction on how to improve their work.\n\n2. Condescending tone: The boss\'s statement, "It\'s not something you\'d understand at your level," is condescending and implies that the team is not capable of understanding or addressing the issues with the proposal.\n\n3. Unwillingness to mentor: When Colleague 2 expresses a willingness to learn and make necessary changes, the boss dismisses this opportunity to guide and mentor the team, instead choosing to take over the task entirely.\n\n4. Discouraging initiative: By telling the team to "focus on your designated areas," the boss discourages them from taking initiative, learning new skills, and growing professionally.\n\n5. Poor leadership: A good leader should provide constructive fe